In [1]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
import chromadb
import pandas as pd

In [5]:
#from transformers import BertTokenizer

In [6]:
from openai import AzureOpenAI, BadRequestError
from configparser import ConfigParser, ExtendedInterpolation
import httpx
import concurrent
from tqdm import tqdm
from tenacity import retry, stop_after_attempt, wait_random_exponential
config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('./monitor-de-iniciativas/openai/config.ini', 'UTF-8')
http_client = httpx.Client(verify='./monitor-de-iniciativas/openai/petrobras-ca-root.pem')
client = AzureOpenAI(
    api_key=config['OPENAI']['OPENAI_API_KEY'],  
    api_version=config['OPENAI']['OPENAI_API_VERSION'],
    # azure_endpoint=config['OPENAI']['OPENAI_API_BASE'],
    base_url=config['OPENAI']['AZURE_OPENAI_BASE_URL'],
    http_client=http_client
)

In [7]:
import uuid
from getpass import getpass
import os
#from pipeline import client
from tenacity import retry, stop_after_attempt, wait_random_exponential


In [8]:
senha = getpass('Senha do proxy:')
os.environ['http_proxy'] = 'http://z2aa:' + senha + '@inet-sys.gnet.petrobras.com.br:8080'
os.environ['https_proxy'] = 'http://z2aa:' + senha + '@inet-sys.gnet.petrobras.com.br:8080'

Senha do proxy: ········


In [9]:
chromadb_path = "./chroma_embeddings"
chroma_client = chromadb.PersistentClient(path=chromadb_path)

In [10]:
collection = chroma_client.create_collection(name="testeHelios")

In [10]:
#collectionn = chroma_client.get_collection("teste2")

In [11]:
# Split a text into smaller chunks of size n, preferably ending at the end of a sentence
def create_chunks(text, n, tokenizer):
    """Returns successive n-sized chunks from provided text."""
    tokens = tokenizer.encode(text)
    i = 0
    while i < len(tokens):
        # Find the nearest end of sentence within a range of 0.5 * n and 1.5 * n tokens
        j = min(i + int(1.5 * n), len(tokens))
        while j > i + int(0.5 * n):
            # Decode the tokens and check for full stop or newline
            chunk = tokenizer.decode(tokens[i:j])
            if chunk.endswith(".") or chunk.endswith("\n"):
                break
            j -= 1
        # If no end of sentence found, use n tokens as the chunk size
        if j == i + int(0.5 * n):
            j = min(i + n, len(tokens))
        yield tokens[i:j]
        i = j


In [12]:
@retry(wait=wait_random_exponential(min=10, max=80), stop=stop_after_attempt(10))
def get_embedding(text):#(model, text):
    """Transform a text in a vector using embedding model"""
    embedding = client.embeddings.create(
        input=text,
        model="text-embedding-petrobras"
    )
    #embedding = model.encode(text)#, convert_to_tensor=True)
    return embedding.data[0].embedding
    #return [float(x) for x in list(embedding)]#.data[0].embedding

In [13]:
def prepare_documents(documents, meta):
    """Prepare documents to vector database, generating embedding and metadata"""
    embeddings = []
    metadatas = []
    docs = []
    for i, doc in enumerate(documents):
        tdoc = tokenizer.decode(doc) #decodifica os tokens do documento usando o objeto tokenizer e armazena o resultado na variável tdoc
        docs.append(tdoc)
        embeddings.append(get_embedding(tdoc))  #obtem vetor de embedding do documento decodificado tdoc e adiciona o resultado à lista embeddings
        metadatas.append({"title": meta['titulo'], "link":meta['url'], "partition" : i}) #adiciona um dicionário contendo as informações de metadados fornecidas (meta['titulo'] e meta['url']) e o índice i à lista metadatas.

    return docs, embeddings, metadatas

In [14]:
def create_ids(documents):
    """Create a list of IDs for documents"""
    return [str(uuid.uuid1()) for _ in documents] #gera uma lista de IDs únicos para cada documento

def insert_data(documents, embeddings, metadatas, ids): #documents (lista de documentos), embeddings (lista de vetores de embedding), metadatas (lista de metadados) e ids (lista de IDs). 
    """Insert data in a ChromaDB collection""" #insere os dados na coleção do ChromaDB usando o método add do objeto collection. 
    collection.add(
        embeddings=embeddings,
        documents=documents,
        metadatas=metadatas,
        ids=ids
    )
    print(f"Data successfully entered! {len(documents)} Chunks") #imprime sucesso e o número de chucks inseridos

In [34]:
def run(df, tokenizer):
    print("Running prep docs...")

    documents = []
    embeddings = []
    metadatas = []
    for _, row in df.iterrows(): 
        document = row['texto']
        document_chunks = create_chunks(document, 512, tokenizer)
        documentlala, document_embeddings, document_metadatas = prepare_documents(document_chunks, row) #preparar as partes do documento, retornando as listas de documentos, vetores de embedding e metadados correspondente
        documents.extend(documentlala) #Os documentos resultantes são estendidos nas listas documents
        embeddings.extend(document_embeddings)
        metadatas.extend(document_metadatas)
    
    ids = create_ids(documents)
    insert_data(documents, embeddings, metadatas, ids) #insere os documentos, vetores de embedding, metadados e IDs na coleção do ChromaDB.

In [35]:
df = pd.read_excel('EPBR_EXCEL_566.xlsx') #lê um arquivo Excel chamado "base_traduzida2.xlsx" e carrega em um DataFrame do Pandas
df.head()

,Unnamed: 0,titulo,veiculo,url,data,descricao,texto,rotulo
0,0,Como a mudança climática piorou a seca na Amaz...,ERP,https://epbr.com.br/como-a-mudanca-climatica-p...,25 de janeiro de 2024 - Atualizado em 29 de ja...,Crise climática tornou 30 vezes mais provável ...,"Em outubro de 2023, o Rio Madeira, na Amazônia...",Não relacionado
1,1,Equinor e Galp têm novos clientes no mercado d...,ERP,https://epbr.com.br/equinor-e-galp-tem-novos-c...,20 de fevereiro de 2024 - Atualizado em 21 de ...,"Norueguesa começa a vender gás no Centro-Sul, ...","RIO – A exemplo da Shell e da Galp, a Equinor ...",Não relacionado
2,2,ANP investiga origens do metanol usado na adul...,ERP,https://epbr.com.br/anp-investiga-origens-do-m...,23 de janeiro de 2024 - Atualizado em 26 de ja...,"A agência está mapeando mercado do solvente, i...",""" BRASÍLIA — As ações de fiscalização da Agênc...",Não relacionado
3,3,Brasil registrou recorde de desastres climátic...,ERP,https://epbr.com.br/brasil-registrou-recorde-d...,23 de janeiro de 2024 - Atualizado em 24 de ja...,"No ano passado, 1.161 ocorrências foram mapead...",Confirmado pelos cientistas como o ano mais qu...,Não relacionado
4,4,Preço do petróleo cai e volta a ficar abaixo d...,ERP,https://epbr.com.br/preco-do-petroleo-cai-e-vo...,24 de janeiro de 2024,"A queda foi provocada, segundo analistas, pela...",Você vai ver por aqui: Preço do petróleo cai ...,Não relacionado


In [36]:
import tiktoken

In [37]:
#model = SentenceTransformer("rufimelo/bert-large-portuguese-cased-sts")
#tokenizer = BertTokenizer.from_pretrained("rufimelo/bert-large-portuguese-cased-sts")
tokenizer = tiktoken.encoding_for_model("gpt-4-32k-petrobras")

In [ ]:
run(df, tokenizer)

Running prep docs...


In [ ]:
def search_document(question):
    """Search for relevant documents in ChromaDB according to the question"""
    query_embedding = get_embedding(question)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=5, include=['distances', 'documents', 'metadatas']
    )
    return results

def format_search_result(relevant_documents):
    """Auxiliary function to format a list of documents into a string"""
    formatted_list = []    
    for i, doc in enumerate(relevant_documents["documents"][0]):
        formatted_list.append("[{}]:\n{}".format(relevant_documents["metadatas"][0][i]["title"], doc))
    
    documents_str = "\n".join(formatted_list)
    return documents_str

In [ ]:
r = search_document("hidrogênio verde")
result_docs = format_search_result(r)
print(result_docs)

In [ ]:
question ="Existiram iniciativas de eólica na Noruega em 2024?"
#, e cite a fonte quando fornecer a informação. 
r = search_document(question)
result_docs = format_search_result(r)
prompt = f"""Voce é um assistente de IA especializado no mercado de energia. Conhece o contexto dos acontecimentos recentes e as tecnologias envolvidas. É prestativo e sucinto nas suas respostas.
Os documentos abaixo apresentam as fontes atualizadas e devem ser consideradas como verdade.
Utilize-as caso necessário.
Documentos:
{result_docs}
"""

In [ ]:
messages=[
  {"role": "system","content": f"{prompt}"},
  {"role": "user","content": f"{question}"}
]

response = client.chat.completions.create(
            model="gpt-4-32k-petrobras",
            temperature=0.0,
            messages=messages
        )

In [ ]:
print(question)
print()
print(prompt)
print()
print(response.choices[0].message.content)